In [1]:
import tensorflow as tf

## 기본 사용법(Basic Usage)

TensorFlow를 사용하기 위해 먼저 TensorFlow가 어떻게 동작하는지를 이해해 봅시다.

- 연산은 graph로 표현합니다.(역자 주: graph는 점과 선, 업계 용어로는 노드와 엣지로 이뤄진 수학적인 구조를 의미합니다.)
- graph는 Session내에서 실행됩니다.
- 데이터는 tensor로 표현합니다.
- 변수(Variable)는 (역자 주: 여러 graph들이 작동할 때도) 그 상태를 유지합니다.
- 작업(operation 혹은 op)에서 데이터를 입출력 할 때 feed와 fetch를 사용할 수 있습니다.


## Tensors

TensorFlow 프로그램은 모든 데이터를 tensor 데이터 구조로 나타냅니다. 

연산 graph에 있는 작업들(op) 간에는 tensor만 주고받을 수 있기 때문입니다. 

TensorFlow의 tensor를 n 차원의 배열이나 리스트라고 봐도 좋습니다. 

tensor는 정적인 타입(static type), 차원(rank 역자 주: 예를 들어 1차원, 2차원하는 차원), 형태(shape, 역자 주: 예를 들어 2차원이면 m x n) 값을 가집니다.

## 연산 graph(The computation graph)

TensorFlow 프로그램은 보통 graph를 조립하는 '구성 단계(construction phase)'와 session을 이용해 graph의 op을 실행시키는 '실행 단계(execution phase)'로 구성됩니다.

예를 들어 뉴럴 네트워크를 표현하고 학습시키기 위해 구성 단계에는 graph를 만들고 실행 단계에는 graph의 훈련용 작업들(set of training ops)을 반복해서 실행합니다.


## graph 만들기(Building the graph)

graph를 만드는 것은 상수(constant)같이 아무 입력값이 필요없는 작업(op)을 정의하는 것에서부터 시작합니다. 이 op을 연산이 필요한 다른 op들에게 입력값으로 제공하는 것입니다.

파이썬 라이브러리의 작업 생성 함수(op constructor)는 만들어진 작업(op)들의 결과값을 반환합니다. 

반환된 작업들의 결과값은 다른 작업(op)을 생성할 때 함수의 입력값으로 이용할 수 있습니다.


기본적인 tensor와 graph를 만들자

In [2]:
m1 = tf.constant([[3.,3.]]) # 상수
m2 = tf.constant([[2.],[2.]])

prod = tf.matmul(m1,m2) # 행렬곱 matmul op의 결과

tf.Tensor 클래스의 attribute와 method 확인하자

In [3]:
type(m1)

tensorflow.python.framework.ops.Tensor

In [5]:
type(prod) # 역시 Tensor 클래스

tensorflow.python.framework.ops.Tensor

In [6]:
m1

<tf.Tensor 'Const:0' shape=(1, 2) dtype=float32>

In [7]:
m1.name

'Const:0'

In [8]:
m1.shape # rank 2(행렬)인 tensor

TensorShape([Dimension(1), Dimension(2)])

In [22]:
tf.rank(m1) # 세션 내부에서 확인해야함(?)

<tf.Tensor 'Rank_4:0' shape=() dtype=int32>

In [9]:
m1.dtype

tf.float32

In [10]:
m1.consumers() # Returns a list of Operations that consume this tensor.

[<tf.Operation 'MatMul' type=MatMul>]

In [11]:
m1.device

''

In [12]:
m1.op

<tf.Operation 'Const' type=Const>

In [13]:
m1.value_index

0

In [15]:
m1.eval() # 세션없기에 평가 불가

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [17]:
with tf.Session() as sess:
    with tf.device("/cpu:1"): # 2번째 cpu 사용을 명시
        print(type(sess)) # Session 클래스
        
        res1 = sess.run(m1) # 그냥 넣으면
        print(res1, type(res1)) # 파이썬에서는 numpy.ndarray를 리턴
        
        res2 = sess.run([m2]) # 반면 리스트로 감싸서 넣으면
        print(res2,type(res2)) # list를 리턴
        
        print(sess.run(prod)) # matmul op의 결과
        
        print(prod.eval()) # 세션내에선 eval 가능

<class 'tensorflow.python.client.session.Session'>
[[3. 3.]] <class 'numpy.ndarray'>
[array([[2.],
       [2.]], dtype=float32)] <class 'list'>
[[12.]]
[[12.]]


### current graph 까보기

tf.get_default_graph():

    Returns the default graph for the current thread.

In [2]:
a = tf.constant(3, name="a_in")
b = tf.add(a,a, name="a_add")

In [3]:
g = tf.get_default_graph()
print(g)

In [4]:
ops = g.get_operations() # 그래프내 op들의 리스트 받아온다
print(ops[0].name)
print(ops[1].name)

a_in
a_add


In [5]:
with tf.Session() as sess:
    print(sess.run(b))

6


In [6]:
print(g.as_default()) # graph 의 contextManager 읽어온다

In [7]:
print(g.as_graph_def()) # 현재 그래프에 정의된 node 출력(각 노드 내에 operation이 1개씩 정의)

node {
  name: "a_in"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "a_add"
  op: "Add"
  input: "a_in"
  input: "a_in"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 26
}



In [8]:
print(g.as_graph_element(a)) # graph 내부 텐서들 본다
print(g.as_graph_element(b))

Tensor("a_in:0", shape=(), dtype=int32)
Tensor("a_add:0", shape=(), dtype=int32)


## operation

Represents a **graph node** that performs computation on tensors.

An Operation is a node in a TensorFlow Graph that **takes zero or more Tensor** objects as input, and **produces zero or more Tensor** objects as output. 

Objects of type Operation are created by calling a Python op constructor (such as tf.matmul) or tf.Graph.create_op.

For example **c = tf.matmul(a, b)** creates an Operation of type "MatMul" that takes tensors a and b as input, and produces c as output.

After the graph has been launched in a session, an Operation can be executed by passing it to **tf.Session.run.op.run()** is a shortcut for calling *tf.get_default_session().run(op)*.

In [9]:
g.get_operation_by_name('a_in') # 그래프에서 by name으로 op 읽어온다

<tf.Operation 'a_in' type=Const>

In [10]:
ops = g.get_operations() # 또는 리스트로 다 가져온다
print(ops)

[<tf.Operation 'a_in' type=Const>, <tf.Operation 'a_add' type=Add>]


In [11]:
ops[0].node_def # op의 노드구조 출력

name: "a_in"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 3
    }
  }
}

In [18]:
print(ops[0].name,'\n\n'
      ,ops[0].type,'\n\n'
      ,ops[0].values(),'\n\n'
      ,ops[0].get_attr("value"),'\n\n'
      ,ops[0].device)

a_in 

 Const 

 (<tf.Tensor 'a_in:0' shape=() dtype=int32>,) 

 dtype: DT_INT32
tensor_shape {
}
int_val: 3
 

 


In [21]:
# inputs ->  The list of `Tensor` objects representing the data inputs of this op.
print(list(ops[0].inputs))
print(list(ops[1].inputs))

[]
[<tf.Tensor 'a_in:0' shape=() dtype=int32>, <tf.Tensor 'a_in:0' shape=() dtype=int32>]


In [23]:
print(ops[0].outputs) # operation이 실행되면 텐서를 구성한다
print(ops[1].outputs)

[<tf.Tensor 'a_in:0' shape=() dtype=int32>]
[<tf.Tensor 'a_add:0' shape=() dtype=int32>]


In [24]:
print(ops[0].graph) # 또한 op는 자신이 속한 그래프 객체를 레퍼런스한다
print(g)

## placeholder 와 variables

In [25]:
# tf.placeholder: 계산을 실행할 때 입력값을 받는 변수로 사용합니다.
# [None,3] 의 의미?? shape를 표시하고 앞에 차원과 상관없이 실제 뒤에 원소가 3개를 말한다.

X = tf.placeholder(tf.float32,[None,3]) 

In [29]:
type(X)

tensorflow.python.framework.ops.Tensor

In [33]:
X

<tf.Tensor 'Placeholder:0' shape=(?, 3) dtype=float32>

In [26]:
# X 플레이스홀더에 넣을 값 입니다.
# 플레이스홀더에서 설정한 것 처럼, 두번째 차원의 요소의 갯수는 3개 입니다.
x_data = [[1, 2, 3], [4, 5, 6]]

In [27]:
# tf.Variable: 그래프를 계산하면서 최적화 할 변수들입니다. 이 값이 바로 신경망을 좌우하는 값들입니다.
# tf.random_normal: 각 변수들의 초기값을 정규분포 랜덤 값으로 초기화합니다.
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

In [31]:
type(W)

tensorflow.python.ops.variables.Variable

In [34]:
W

<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32_ref>

In [35]:
# 입력값과 변수들을 계산할 수식을 작성합니다.
# tf.matmul 처럼 mat* 로 되어 있는 함수로 행렬 계산을 수행합니다.
expr = tf.matmul(X, W) + b

In [36]:
with tf.Session() as sess:
    # 위에서 설정한 Variable 들의 값들을 초기화 하기 위해
    # 처음에 tf.global_variables_initializer 를 한 번 실행해야 합니다.
    sess.run(tf.global_variables_initializer())
    sess.run(W)
    sess.run(b)
    
    # expr 수식에는 X 라는 입력값이 필요합니다.
    # 따라서 expr 실행시에는 이 변수에 대한 실제 입력값을 다음처럼 넣어줘야합니다.
    print(sess.run(expr, feed_dict={X: x_data}))

[[ 3.2004044 -1.7570053]
 [ 8.493158  -3.489602 ]]


변수 객체를 다뤄보자

In [45]:
# 값이 0인 스칼라로 초기화된 변수를 만듭니다.
state = tf.Variable(0, name="counter")

In [46]:
# 'state'에 1을 더하는 작업(op)을 만듭니다.
# assign 메소드로 update 합니다. 역시 그래프의 한 부분이므로 세션에서 run해야 실제로 작동합니다.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

In [47]:
# 그래프를 한 번 작동시킨 후에는 'init' 작업(op)을 실행해서 변수를 초기화해야
# 합니다. 먼저 'init' 작업(op)을 추가해 봅시다.

init_op = tf.global_variables_initializer()

In [49]:
# graph와 작업(op)들을 실행시킵니다.
with tf.Session() as sess:
  # 'init' 작업(op)을 실행합니다.
  sess.run(init_op)
    
  # 'state'의 시작값을 출력합니다.
  print('시작값:',sess.run(state))
    
  # 'state'값을 업데이트하고 출력하는 작업(op)을 실행합니다.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))

시작값: 0
1
2
3
